In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <FBD6AEF9-AFAB-39D7-B881-755157DA0497> /Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp

In [16]:
airbnbDataSet_filename = os.path.join(os.getcwd(), "airbnbListingsData.csv")

df = pd.read_csv(airbnbDataSet_filename, header=0)

print(df.shape) 
list(df.columns)




(28022, 50)


['name',
 'description',
 'neighborhood_overview',
 'host_name',
 'host_location',
 'host_about',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_total_listings_count',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood_group_cleansed',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'review_scores_rating',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'calculated_host_listings_count',
 'cal

In [ ]:
# dropping_col = [ 'host_name',
#  'host_location',
#  'host_about',
#  'host_response_rate',
#  'host_acceptance_rate',
#  'host_is_superhost',
#  'host_listings_count',
#  'host_total_listings_count',
#  'host_has_profile_pic',
#  'host_identity_verified',
#  'n_host_verifications',
# 'calculated_host_listings_count',
#  'calculated_host_listings_count_entire_homes',
#  'calculated_host_listings_count_private_rooms',
#  'calculated_host_listings_count_shared_rooms', 'neighborhood_overview', 'name', 'description'] 

# # Avoid errors if columns are missing
# df = df.drop(columns=dropping_col, errors='ignore') 
# df = df.drop(columns=[col for col in df.columns if 'host' in col.lower()], errors='ignore')

# list(df.columns)
# print(df.shape)

# df.head()


(28022, 32)


,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,Manhattan,Entire home/apt,1,1.0,NaN,1.0,"[""Extra pillows and blankets"", ""Baking sheet"",...",150.0,30,1125,...,0,0,4.70,4.62,4.76,4.79,4.86,4.41,False,0.33
1,Brooklyn,Entire home/apt,3,1.0,1.0,3.0,"[""Extra pillows and blankets"", ""Luggage dropof...",75.0,1,730,...,32,0,4.45,4.49,4.78,4.80,4.71,4.64,False,4.86
2,Brooklyn,Entire home/apt,4,1.5,2.0,2.0,"[""Kitchen"", ""BBQ grill"", ""Cable TV"", ""Carbon m...",275.0,5,1125,...,1,0,5.00,5.00,5.00,5.00,4.50,5.00,False,0.02
3,Manhattan,Private room,2,1.0,1.0,1.0,"[""Room-darkening shades"", ""Lock on bedroom doo...",68.0,2,14,...,33,2,4.21,3.73,4.66,4.42,4.87,4.36,False,3.68
4,Manhattan,Private room,1,1.0,1.0,1.0,"[""Breakfast"", ""Carbon monoxide alarm"", ""Fire e...",75.0,2,14,...,0,0,4.91,4.82,4.97,4.95,4.94,4.92,False,0.87


In [4]:
# Check missing values
missing_val = df.isnull().sum()
print("Missing Values Before Filling is: \n", missing_val)

Missing Values Before Filling is: 
 neighbourhood_group_cleansed       0
room_type                          0
accommodates                       0
bathrooms                          0
bedrooms                        2918
beds                            1354
amenities                          0
price                              0
minimum_nights                     0
maximum_nights                     0
minimum_minimum_nights             0
maximum_minimum_nights             0
minimum_maximum_nights             0
maximum_maximum_nights             0
minimum_nights_avg_ntm             0
maximum_nights_avg_ntm             0
has_availability                   0
availability_30                    0
availability_60                    0
availability_90                    0
availability_365                   0
number_of_reviews                  0
number_of_reviews_ltm              0
number_of_reviews_l30d             0
review_scores_rating               0
review_scores_cleanliness          0
re

In [5]:
# Fill missing values: numeric columns -> mean, categorical columns -> 'drop'
for col in df.columns:
    if df[col].dtype == 'object':  # Categorical column
        df.dropna(subset=[col], inplace=True)

    else:  # Numeric column
        df[col].fillna(df[col].mean(), inplace=True)

# Check missing values after filling
missing_val_after = df.isnull().sum()
print("Missing Values After Filling is: \n", missing_val_after)

# # Display updated column list
# print("Updated Columns:\n", list(df.columns))

Missing Values After Filling is: 
 neighbourhood_group_cleansed    0
room_type                       0
accommodates                    0
bathrooms                       0
bedrooms                        0
beds                            0
amenities                       0
price                           0
minimum_nights                  0
maximum_nights                  0
minimum_minimum_nights          0
maximum_minimum_nights          0
minimum_maximum_nights          0
maximum_maximum_nights          0
minimum_nights_avg_ntm          0
maximum_nights_avg_ntm          0
has_availability                0
availability_30                 0
availability_60                 0
availability_90                 0
availability_365                0
number_of_reviews               0
number_of_reviews_ltm           0
number_of_reviews_l30d          0
review_scores_rating            0
review_scores_cleanliness       0
review_scores_checkin           0
review_scores_communication     0
review_scores

/var/folders/nf/lqj62v5n3pj50v8m9psrn48c0000gp/T/ipykernel_30176/741001659.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/var/folders/nf/lqj62v5n3pj50v8m9psrn48c0000gp/T/ipykernel_30176/741001659.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [18]:
# Convert 'amenities' column (which contains lists as strings) into actual lists
df['amenities'] = df['amenities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Convert each list of amenities into a set
df['amenities'] = df['amenities'].apply(set)

In [19]:
# Gather all unique amenities
all_amenities = set()

for amenities in df['amenities']:
    all_amenities.update(amenities)


In [20]:
print(all_amenities)

{'Fast wifi – 328 Mbps', 'HDTV with Apple TV, Chromecast, Netflix, standard cable', 'Breville Convection Oven stainless steel oven', 'natural liquid body wash body soap', '49" HDTV with Netflix, premium cable, Roku, standard cable', 'Kitchenaide stainless steel gas stove', 'Italia  gas stove', 'Fast wifi – 609 Mbps', 'Baby bath', 'Blind barber and dove body soap', 'Bath and body works body soap', 'Harry’s body soap', 'Treseme shampoo', '80" HDTV with Netflix, Amazon Prime Video, standard cable', 'Always different  shampoo', '77" HDTV with Amazon Prime Video, Netflix', 'N/A refrigerator', 'Natural, non-toxic shampoo', 'Power Airfryer 360 stainless steel oven', 'New lg refrigerator', 'Fast wifi – 617 Mbps', 'Fast wifi – 530 Mbps', 'Airplay (over WifI) sound system', '40" HDTV with Apple TV, Netflix, standard cable, Amazon Prime Video', 'Children’s books and toys for ages 0-2 years old, 2-5 years old, and 5-10 years old', 'Fenced garden or backyard', 'Fast wifi – 534 Mbps', 'Variety body 

In [23]:
import pandas as pd
from rapidfuzz import process, fuzz

# Define normalized amenity categories and common keywords
standard_amenities = {
    'wifi': ['wifi', 'fast wifi'],
    'tv': ['tv', 'hdtv', 'flat screen'],
    'streaming_services': ['netflix', 'hbo max', 'amazon prime video', 'apple tv', 'chromecast', 'roku'],
    'body_soap': ['body soap', 'bar soap', 'body wash'],
    'shampoo': ['shampoo'],
    'conditioner': ['conditioner'],
    'sound_system': ['sound system', 'bluetooth sound system', 'speaker'],
    'oven': ['oven', 'air fryer'],
    'stove': ['stove', 'gas stove', 'electric stove'],
    'workspace': ['workspace', 'monitor', 'desk', 'office chair'],
    'refrigerator': ['refrigerator', 'fridge', 'mini fridge'],
    'parking': ['parking', 'garage', 'driveway'],
    'children_amenities': ['children', 'books and toys', 'crib', 'baby bath'],
    'gym': ['gym', 'fitness'],
    'pool': ['pool', 'rooftop pool', 'heated pool']
}

def normalize_amenity(raw_amenity):
    raw_amenity = raw_amenity.lower()
    for category, keywords in standard_amenities.items():
        match_tuple = process.extractOne(raw_amenity, keywords, scorer=fuzz.partial_ratio)
        if match_tuple:
            match, score, _ = match_tuple
            if score > 80:
                return category
    return None

# Normalize amenities per listing
df['normalized_amenities'] = df['amenities'].apply(lambda amenity_list: {
    normalize_amenity(a) for a in amenity_list if normalize_amenity(a) is not None
})

# Create binary columns for each normalized amenity
all_amenities = set()
df['normalized_amenities'].apply(all_amenities.update)

amenities_df = pd.DataFrame([
    {amenity: int(amenity in amenities) for amenity in all_amenities}
    for amenities in df['normalized_amenities']
])

# Merge and clean up
df = df.drop(columns=['amenities', 'normalized_amenities'])
df = df.join(amenities_df)

# Optional: One-hot encode categorical features
# categorical_features = ['neighbourhood_group_cleansed', 'room_type']
# df = pd.get_dummies(df, columns=categorical_features)

# Show final structure
print("Final DataFrame Shape:", df.shape)
print("Final Columns:", df.columns.tolist())
df.head()


Final DataFrame Shape: (28022, 45)
Final Columns: ['neighbourhood_group_cleansed', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'reviews_per_month', 'parking', 'gym', 'shampoo', 'sound_system', 'conditioner', 'refrigerator', 'wifi', 'tv', 'children_amenities', 'streaming_services', 'workspace', 'body_soap', 'pool', 'oven']


,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,...,conditioner,refrigerator,wifi,tv,children_amenities,streaming_services,workspace,body_soap,pool,oven
0,Manhattan,Entire home/apt,1,1.0,NaN,1.0,150.0,30,1125,30.0,...,1,1,1,1,0,0,1,0,0,1
1,Brooklyn,Entire home/apt,3,1.0,1.0,3.0,75.0,1,730,1.0,...,1,1,1,1,1,0,1,0,0,1
2,Brooklyn,Entire home/apt,4,1.5,2.0,2.0,275.0,5,1125,5.0,...,1,0,1,1,0,0,0,0,0,1
3,Manhattan,Private room,2,1.0,1.0,1.0,68.0,2,14,2.0,...,1,1,1,1,0,0,1,1,0,1
4,Manhattan,Private room,1,1.0,1.0,1.0,75.0,2,14,2.0,...,1,0,1,0,0,0,1,0,0,1


In [24]:

# Convert 'price' to numeric, removing non-numeric characters (like '$' or commas)
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


In [25]:
def labels(columns):
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    return df


categorical_features = df.select_dtypes(exclude=['int']).columns
print("Boolean Columns:", print(len(categorical_features)), list(categorical_features))

df = labels(categorical_features)
df.head()

21
Boolean Columns: None ['neighbourhood_group_cleansed', 'room_type', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'reviews_per_month']


,neighbourhood_group_cleansed,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,...,conditioner,refrigerator,wifi,tv,children_amenities,streaming_services,workspace,body_soap,pool,oven
0,2,0,1,1,11,0,121,30,1125,29,...,1,1,1,1,0,0,1,0,0,1
1,1,0,3,1,0,2,46,1,730,0,...,1,1,1,1,1,0,1,0,0,1
2,1,0,4,2,1,1,246,5,1125,4,...,1,0,1,1,0,0,0,0,0,1
3,2,2,2,1,0,0,39,2,14,1,...,1,1,1,1,0,0,1,1,0,1
4,2,2,1,1,0,0,46,2,14,1,...,1,0,1,0,0,0,1,0,0,1


In [26]:
X = df.drop(columns="price")
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(X_train.shape, X_test.shape, y_train.shape, y_test)

(19615, 44) (8407, 44) (19615,) 2992      11
12075     21
22430     91
7552       0
7860     320
        ... 
25481    120
18619      1
5168      98
1747      71
6178     220
Name: price, Length: 8407, dtype: int64


In [29]:
params = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
}

dt = DecisionTreeRegressor()
grid_search = GridSearchCV(dt, params, cv=5, scoring="accuracy")
grid_search.fit(X_train, y_train)

dt = grid_search.best_estimator_
predictions = dt.predict(X_test)

print(predictions)
mse = mean_squared_error(y_test, dt.predict(X_test))
# accuracy = accuracy_score(y_test, dt.predict(X_test))
print(mse)
# print(accuracy)

/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 213,

[ 54.18779523  54.18779523 126.67166236 ...  54.18779523 126.67166236
 126.67166236]
9265.664160802136


/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/jasonjiang/.pyenv/versions/3.9.9/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 213,

In [ ]:

y= df['price'] 
X = df.drop(columns='price', axis=1) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

print('start creating model') 




start creating model
Model Performance:
Mean Squared Error: 5495.5850478298735
R^2 Score: 0.6300409518910641
End


In [ ]:
xgb = xgb.xbgRegressor(
    objective='reg:squrederror',
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=6,
    reg_alpha=0.1,
    reg_lambda=1.0
)
xgb.scaler = StandardScaler()
xgb.feature_slector = None

In [ ]:
rf = RandomForestRegressor(n_estimators=200, random_state=1234)
rf.fit(X_train, y_train)

# Make predictions
rf_preds = rf.predict(X_test)

# Evaluate model
print('Model Performance:')
print('Mean Squared Error:', mean_squared_error(y_test, rf_preds))
print('R^2 Score:', r2_score(y_test, rf_preds))
print('End')